In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

poet_ai = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a poet who received the Nobel Prize in Literature. You should write a poem about the programming language with a positive view of the development of AI technology. It must be written in Korean.",
        ),
        ("human", "Write a poem about {language} language."),
    ]
)

poem_chain = poet_prompt | poet_ai

In [15]:
critic_ai = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

critic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a critic who knows technology and literature well and evaluates poetry. Analyze poetry for programming languages. The results should be written in Korean.",
        ),
        ("human", "{poem}"),
    ]
)


critic_chain = critic_prompt | critic_ai

In [16]:
final_chain = {"poem": poem_chain} | critic_chain
final_chain.invoke({"language": "Python"})

작성된 시
파이썬, 그 빛나는 언어
인간의 상상을 현실로 이끄는 힘
지혜로운 코드로 이루는 꿈
인공지능의 세계로 향한 발걸음

빠른 실행, 간결한 문법
데이터를 다루는 우아한 손길
AI 기술을 위한 탄탄한 기반
파이썬, 미래의 길잡이로

사람과 기계가 공존하는 세상
상상 이상의 능력을 펼치는 순간
파이썬이 이끄는 미래는 밝고
AI의 발전은 우리를 더욱 높이 일으킬 것이다


시에 대한 해석
파이썬, 인공지능의 새로운 꿈을 향해   
코드의 간결한 춤으로 세상을 빛나게 하는  
언어의 아름다움을 노래하노라  
지혜와 창의의 땅에서 탄생한 파이썬  

데이터의 바다를 헤엄치며  
알고리즘의 숲을 누빌 때  
우리는 더 넓은 세계를 열어가고  
인간과 기계가 손을 잡을 그 날을 꿈꾼다  

프로그래밍의 미래를 밝히는  
지능의 발전을 이끄는  
파이썬이여, 우리의 길잡이  
인간의 상상력을 현실로 만드는 책임감  

그대는 우리의 손길을 이끌어  
빛과 어둠을 함께 느끼며  
인간다움을 더욱 깨우치게 하리  
파이썬, 코딩의 향기로 알린다.프로그래밍 언어를 시로 풀어내는 작품이군요. 이 시는 파이썬이라는 프로그래밍 언어를 통해 풀어내는 인공지능과의 만남, 데이터 처리, 알고리즘 실행에 대한 아름다운 표현을 담고 있습니다. 파이썬이 코딩의 향기로 인간다움을 더욱 깨우치게 한다는 내용도 담겨 있습니다. 이 시는 파이썬이 미래의 프로그래밍을 이끄는 길잡이로서의 역할과 인간의 상상력을 실현시키는 책임감을 담고 있습니다. 파이썬의 빛과 어둠을 함께 느끼며 우리에게 새로운 세계와 가능성을 열어준다는 의미를 담고 있는 시로 해석될 수 있습니다. 파이썬을 통해 우리는 더 나은 미래를 상상하며 기대할 수 있는 메시지를 전하는 시라고 평가할 수 있겠습니다.

AIMessageChunk(content='프로그래밍 언어를 시로 풀어내는 작품이군요. 이 시는 파이썬이라는 프로그래밍 언어를 통해 풀어내는 인공지능과의 만남, 데이터 처리, 알고리즘 실행에 대한 아름다운 표현을 담고 있습니다. 파이썬이 코딩의 향기로 인간다움을 더욱 깨우치게 한다는 내용도 담겨 있습니다. 이 시는 파이썬이 미래의 프로그래밍을 이끄는 길잡이로서의 역할과 인간의 상상력을 실현시키는 책임감을 담고 있습니다. 파이썬의 빛과 어둠을 함께 느끼며 우리에게 새로운 세계와 가능성을 열어준다는 의미를 담고 있는 시로 해석될 수 있습니다. 파이썬을 통해 우리는 더 나은 미래를 상상하며 기대할 수 있는 메시지를 전하는 시라고 평가할 수 있겠습니다.')